In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from mxnet import nd
import os
from sklearn.model_selection import train_test_split


In [2]:
data_path = '../../Data-EPL/data/'
ls = os.listdir(data_path)
li = []
for file in ls:
    if file.endswith('.csv'):
        df = pd.read_csv(data_path+file, index_col=None, header=0)
        li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2018-08-10,Man United,Leicester,2,1,H,1,0,H,A Marriner,...,6,4,11,8,2,5,2,1,0,0
1,2018-08-11,Bournemouth,Cardiff,2,0,H,1,0,H,K Friend,...,4,1,11,9,7,4,1,1,0,0
2,2018-08-11,Fulham,Crystal Palace,0,2,A,0,1,A,M Dean,...,6,9,9,11,5,5,1,2,0,0
3,2018-08-11,Huddersfield,Chelsea,0,3,A,0,2,A,C Kavanagh,...,1,4,9,8,2,5,2,1,0,0
4,2018-08-11,Newcastle,Tottenham,1,2,A,1,2,A,M Atkinson,...,2,5,11,12,3,5,2,2,0,0


# Data Types and Category
The fist thing we need to do is to convert all data to floating point.There are several data categories in this dataset. We shall deal with each type differently.
- **Date:** 
- **Teams:** This is string that we need to either turn into a one-hot vector or we might decide to turn into absolute numbers representing an index. ***Before habitually converting every feature into a dataset, we should need to investigate whether the field is actually necessary.*** We shall elaborate later.
- **Numerical fields:** Numbers do signify a scale. For instance half-time score could have a bearing in the result of a game.
- **Categorical:** HTR represents three options as which team has won the half-time, away, home, or it has been a draws. We normally convert such features into one-hot encodings. 

## String
- Convert string to wither lower or upper case
- Convert to one-hot

In [3]:
# Convert Cast
df['HomeTeam'] = df['HomeTeam'].str.lower()
df['AwayTeam'] = df['AwayTeam'].str.lower()
df['HomeTeam'] = df['HomeTeam'].str.lower()
df['Referee'] = df['Referee'].str.lower()
df_orig = df.copy()

df.head()


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2018-08-10,man united,leicester,2,1,H,1,0,H,a marriner,...,6,4,11,8,2,5,2,1,0,0
1,2018-08-11,bournemouth,cardiff,2,0,H,1,0,H,k friend,...,4,1,11,9,7,4,1,1,0,0
2,2018-08-11,fulham,crystal palace,0,2,A,0,1,A,m dean,...,6,9,9,11,5,5,1,2,0,0
3,2018-08-11,huddersfield,chelsea,0,3,A,0,2,A,c kavanagh,...,1,4,9,8,2,5,2,1,0,0
4,2018-08-11,newcastle,tottenham,1,2,A,1,2,A,m atkinson,...,2,5,11,12,3,5,2,2,0,0


In [4]:
# Creating a list of teams in a given season. Using 'set'  data tyoes enforeces uniqueness and eliminated duplicate
# team names. We thus create an reference list for teams that are involved in a season.
teams = list(set(df['HomeTeam']))
num_teams = len(teams)
num_teams

36

In [5]:
home_teams = list(df["HomeTeam"].values)
ht0b = home_teams[0]
('HomeTeam value for the firt row in the dataset', ht0b)

('HomeTeam value for the firt row in the dataset', 'man united')

In [6]:
# We now replace teams names with their numerical index.
for i in range(len(home_teams)):
    home_teams[i] = teams.index(home_teams[i])
ht0a = home_teams[0]
('HomeTeam index for the firt row in the dataset', ht0a)

('HomeTeam index for the firt row in the dataset', 33)

In [7]:
#Optionally we can replace the column in the dataframe with the index array or add a new colunm if we choose to.
REPLACE = True
if REPLACE:
        df["HomeTeam"] = home_teams
else:
    df["HomeTeam_idx"] = home_teams
    #Optionally dropping the column
    #df = df.drop(columns="HomeTeam")
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2018-08-10,33,leicester,2,1,H,1,0,H,a marriner,...,6,4,11,8,2,5,2,1,0,0
1,2018-08-11,13,cardiff,2,0,H,1,0,H,k friend,...,4,1,11,9,7,4,1,1,0,0
2,2018-08-11,16,crystal palace,0,2,A,0,1,A,m dean,...,6,9,9,11,5,5,1,2,0,0
3,2018-08-11,3,chelsea,0,3,A,0,2,A,c kavanagh,...,1,4,9,8,2,5,2,1,0,0
4,2018-08-11,23,tottenham,1,2,A,1,2,A,m atkinson,...,2,5,11,12,3,5,2,2,0,0


In [8]:
# Finally we use mxnet.nd.one_hot, in order to produce one-hot vectors for club names.
home_teams_1hot = nd.one_hot(nd.array(home_teams), num_teams)
home_teams_1hot[0]
# In the example below it can be observed that 


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
<NDArray 36 @cpu(0)>

### Challenge
- Convert away team, referee, and HTR into one-hot encoding.

## Do we need team names?
One idea is that the history of a team could be a factor in their results against some other teams. It is reasonable to assume there is pressure of history or even superstition involved in mental preparation of teams. A recent example is when FC Roma banned players to take photos with the rapper Drake (https://www.bbc.com/news/newsbeat-47947155) or the fact that France has usually beaten Brazil in their head to head games usually created a bit of extra pressure.
So we need to frame the problem and figure out what it is that we want to answer. 
- If the idea is to predict **at half time** the result of a game as well as number of corners, fouls, and other regression tasks, then we, might not need to know who is playing.
- If we want to make the same prediction before the game, then we need to know who is playing.

In [9]:
df = df_orig.copy()
df = df.drop(columns=["Date", "HomeTeam", "AwayTeam", "Referee"])
df.head()

,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2,1,H,1,0,H,8,13,6,4,11,8,2,5,2,1,0,0
1,2,0,H,1,0,H,12,10,4,1,11,9,7,4,1,1,0,0
2,0,2,A,0,1,A,15,10,6,9,9,11,5,5,1,2,0,0
3,0,3,A,0,2,A,6,13,1,4,9,8,2,5,2,1,0,0
4,1,2,A,1,2,A,15,15,2,5,11,12,3,5,2,2,0,0


In [10]:
# Shuffling a pandas data frame. It will not be necessary in this case since we will be using 
# sklearn.model_selection.split later. I have left the code here just as a refresher.
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,0,2,A,0,1,A,23,9,12,5,9,17,14,2,1,2,0,0
1,2,1,H,0,0,D,21,5,5,3,4,13,12,4,2,2,0,1
2,0,0,D,0,0,D,17,8,3,0,10,15,11,0,2,3,0,1
3,1,1,D,0,0,D,19,14,4,5,4,11,13,6,1,2,0,0
4,1,1,D,1,0,H,9,13,1,2,18,14,4,6,4,1,0,1


In [11]:
# Converting FTR from 'H', 'A', 'D' to category codes 0, 1, 2
df.FTR = pd.Categorical(df.FTR)
df.FTR = df.FTR.cat.codes

df.HTR = pd.Categorical(df.HTR)
df.HTR = df.HTR.cat.codes

df.head()

,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,0,2,0,0,1,0,23,9,12,5,9,17,14,2,1,2,0,0
1,2,1,2,0,0,1,21,5,5,3,4,13,12,4,2,2,0,1
2,0,0,1,0,0,1,17,8,3,0,10,15,11,0,2,3,0,1
3,1,1,1,0,0,1,19,14,4,5,4,11,13,6,1,2,0,0
4,1,1,1,1,0,2,9,13,1,2,18,14,4,6,4,1,0,1


In [12]:
train, test = train_test_split(df, test_size=0.2)
(df["FTHG"].count(), train["FTHG"].count(), test["FTHG"].count())

(3766, 3012, 754)

In [13]:
train.head()

,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
319,2,0,2,1,0,2,14,16,5,11,15,10,5,8,0,1,0,0
298,2,0,2,1,0,2,17,10,12,4,10,14,11,1,0,2,0,2
2672,3,0,2,0,0,1,23,10,12,0,11,8,6,3,3,2,0,0
1629,2,2,1,0,1,0,24,16,14,10,8,14,4,4,2,2,0,0
3486,2,1,2,0,0,1,7,13,2,7,15,12,4,10,3,2,0,0


In [14]:
train_label = train["FTR"]
train_data = train.drop(columns="FTR")
test_label = test["FTR"]
test_data = test.drop(columns="FTR")

(train_data["FTHG"].count(), train_label.count(), test_data["FTHG"].count(), test_label.count())

(3012, 3012, 754, 754)

In [15]:
train_data = train_data.values
train_label = train_label.values

test_data = test_data.values
test_label = test_label.values

(train_data.shape, train_label.shape, test_data.shape, test_label.shape)


((3012, 17), (3012,), (754, 17), (754,))

In [16]:
np.save(file='data/train_data', arr=train_data)
np.save(file='data/train_label', arr=train_label)
np.save(file='data/test_data', arr=test_data)
np.save(file='data/test_label', arr=test_label)
os.listdir('data')

['test_label.npy', 'train_data.npy', 'train_label.npy', 'test_data.npy']